# ノートブックインスタンスで scikit-learn の学習・推論を行う

#### ノートブックに含まれる内容

- scikit-learn を使って，ローカルで決定木モデルの学習・推論を行うときの基本的なやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: DecisionTreeClassifier
- データ: iris

## セットアップ

In [ ]:
import os
import json
import itertools
import pandas as pd
from sklearn import tree

In [ ]:
# パスをセットして確認
prefix = os.getcwd() 
training_path = os.path.join(prefix, 'data')
param_path = os.path.join(prefix, 'config/hyperparameters.json')

print("training path:  " + training_path)
print("parameter path: " + param_path)

## データのロード

ローカルにあるデータとハイパーパラメタの設定ファイルを読み込みます

In [ ]:
# 学習用データをロード
input_files = [os.path.join(training_path, file) for file in os.listdir(training_path)]
if len(input_files) == 0:
    raise ValueError(('There are no files in {}.').format(training_path))
raw_data = [pd.read_csv(file, header=None) for file in input_files]
train_data = pd.concat(raw_data)

# データの最初のカラムはラベルなので切り分ける
train_y = train_data.iloc[:,[0]]
train_X = train_data.iloc[:,[1,2,3,4]]

print(train_y.shape)
print(train_X.shape)

In [ ]:
# 学習ジョブ用のハイパーパラメタを読み出す
with open(param_path, 'r') as tc:
    trainingParams = json.load(tc)

# 読みだしたハイパーパラメタをセット
# ハイパーパラメタは単一の値のみをサポート
# 常に String で読みだされるため，適切な型変換が必要
max_leaf_nodes = trainingParams.get('max_leaf_nodes', None)
if max_leaf_nodes is not None:
    max_leaf_nodes = int(max_leaf_nodes)

print(max_leaf_nodes)

## モデルの学習を実行

読み込んだデータとハイパーパラメタを用いて，学習用データから予測モデルを構築します

In [ ]:
# scikit-learn で Decision Tree を実行
classifier = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
classifier = classifier.fit(train_X, train_y)

## モデルの推論を実行

構築したモデルに対してデータを食わせて，実際にあやめの種類の推論を行います

In [ ]:
# 推論用のデータを準備
test_X = pd.DataFrame([[5.0, 3.2, 1.2, 4.3], [4.5, 2.3, 1.3, 0.3], [5.7, 2.8, 4.1, 1.3]])

print(test_X)

In [ ]:
# 推論を実行して，結果を表示
results = classifier.predict(test_X)
for i, result in enumerate(results):
    print('predict result of data[{}] is "{}"'.format(i, result))